In [1]:
!pip install torch transformers
!pip install -U datasets
!git clone https://github.com/davidjurgens/citation-function


   ---------------------------------------- 0/6 [sniffio]
   ------ --------------------------------- 1/6 [h11]
   ------ --------------------------------- 1/6 [h11]
   ------ --------------------------------- 1/6 [h11]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------------------- 3/6 [anyio]
   -------------------- ------

fatal: destination path 'citation-function' already exists and is not an empty directory.


In [2]:
import os
with open('api_openrouter.txt', 'r') as f:
  API=f.read()


In [3]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

c:\Users\chrif\Downloads\synthdata1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
for text,intent in zip(test['text'],test['intent']):
    print('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
            + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
    print(text+'\n\n')

You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:From the following list: 0:BACKGROUND 
 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE

Resnik ( 1995 ) reported a correlation of r = .9026.10 The results are not directly comparable , because he only used noun-noun pairs , words instead of concepts , a much smaller dataset , and measured semantic similarity instead of semantic relatedness .


You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the int

In [5]:
dico_rez={
    0:[],
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
}

In [6]:
PROMPT=("""You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:
          "BACKGROUND":"to provide relevant information for this domain (BACKGROUND).",
          "MOTIVATION":"to illustrate need for data, goals, methods, etc.(MOTIVATION)",
          "USES":" to use data, methods, etc from the citation. (USES) ",  
          "CONTINUATION":"to extend the citation's data, methods, etc. (CONTINUATION)",
          "COMPARES OR CONTRASTS":"to express similarity/differences to the citation (COMPARES OR CONTRASTS)",
          "FUTURE": "to be a potential avenue for future work (FUTURE)"
          
          ANSWER WITH THE CORRECT ASSOCIATED NUMBER: 0:BACKGROUND \n 1:USES \n 2:COMPARES OR CONTRASTS \n 3:MOTIVATION \n 4:CONTINUATION \n 5:FUTURE\n"""+ text+'\n\n')

In [7]:
import requests
import json
import os

les_rez=[]
dico_rez={0:[],1:[],2:[],3:[],4:[],5:[]}

url = "https://openrouter.ai/api/v1/chat/completions"


headers = {
  "Authorization": f"Bearer {API}",
  "Content-Type": "application/json"
}


for text,intent in zip(test['text'],test['intent']):
  PROMPT=("""You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation @@CITATION into one of these categories.Think as much as you want, in the end answer only with the associated number:
          "BACKGROUND":"to provide relevant information for this domain (BACKGROUND).",
          "MOTIVATION":"to illustrate need for data, goals, methods, etc.(MOTIVATION)",
          "USES":" to use data, methods, etc from the citation. (USES) ",  
          "CONTINUATION":"to extend the citation's data, methods, etc. (CONTINUATION)",
          "COMPARES OR CONTRASTS":"to express similarity/differences to the citation (COMPARES OR CONTRASTS)",
          "FUTURE": "to be a potential avenue for future work (FUTURE)"
          
          ANSWER WITH THE CORRECT ASSOCIATED NUMBER: 0:BACKGROUND \n 1:USES \n 2:COMPARES OR CONTRASTS \n 3:MOTIVATION \n 4:CONTINUATION \n 5:FUTURE\n"""+ text+'\n\n')
  try:
    payload = {
        "model": "deepseek/deepseek-r1-0528",
      "messages": [
        {
          "role": "user",
          "content": PROMPT
        }
      ]
        ,
    #   "provider": {
    #     "allow_fallbacks": True,
    #     "quantizations": [
    #       "fp8"
    #     ]
    # },
        "reasoning": {

      "enabled": True  # Use high reasoning effort

  }
    }
    response = requests.post(url, headers=headers, json=payload)
    les_rez.append(response.json())
    dico_rez[int(intent)].append(response.json())
  except:
    print("error")
    continue
  print(response.json())

{'id': 'gen-1760097886-8Ar43LUBW8XC5lpwXKAz', 'provider': 'SiliconFlow', 'model': 'deepseek/deepseek-r1-0528', 'object': 'chat.completion', 'created': 1760097886, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '\n2', 'refusal': None, 'reasoning': '\nHmm, the user is asking me to classify the intent of a citation using scientometric categories. I have to determine the category for this specific citation: "Resnik (1995) reported a correlation of r = .9026.10 The results are not directly comparable, because he only used noun-noun pairs, words instead of concepts, a much smaller dataset, and measured semantic similarity instead of semantic relatedness."\n\nLooking at the possible intents:\n- BACKGROUND is for general domain info.\n- MOTIVATION shows need for data/methods.\n- USES indicates using someone else\'s work directly.\n- CONTINUATION is when work extends others\' data/methods.\n- COMPAR

In [9]:
les_rez

[{'id': 'gen-1760097886-8Ar43LUBW8XC5lpwXKAz',
  'provider': 'SiliconFlow',
  'model': 'deepseek/deepseek-r1-0528',
  'object': 'chat.completion',
  'created': 1760097886,
  'choices': [{'logprobs': None,
    'finish_reason': 'stop',
    'native_finish_reason': 'stop',
    'index': 0,
    'message': {'role': 'assistant',
     'content': '\n2',
     'refusal': None,
     'reasoning': '\nHmm, the user is asking me to classify the intent of a citation using scientometric categories. I have to determine the category for this specific citation: "Resnik (1995) reported a correlation of r = .9026.10 The results are not directly comparable, because he only used noun-noun pairs, words instead of concepts, a much smaller dataset, and measured semantic similarity instead of semantic relatedness."\n\nLooking at the possible intents:\n- BACKGROUND is for general domain info.\n- MOTIVATION shows need for data/methods.\n- USES indicates using someone else\'s work directly.\n- CONTINUATION is when wor

In [10]:
with open(f'DATASET9/results.txt', 'a+',encoding='utf-8') as f:
    for elt in les_rez:
        f.write(elt['choices'][0]['message']['content'] + '\n\n')

#dump the json from les_rez
with open(f'DATASET9/results.json', 'w+',encoding='utf-8') as f:
    json.dump(les_rez, f, ensure_ascii=False, indent=4)


In [ ]:
for i in range(0,6):
    for elt in dico_rez[i]:
        with open(f'DATASET7/results_openrouter_intent_{i}.txt', 'a+',encoding='utf-8') as f:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')

In [7]:
with open("DATASET7/dico.json", "w+", encoding="utf-8") as f:
    json.dump(dico_rez, f, ensure_ascii=False, indent=4)